In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:

import os
import sys
os.system('git clone https://github.com/tikendraw/caption-generator.git -q')
os.chdir('caption-generator')


if not os.path.exists('funcyou'):
	os.system('git clone https://github.com/tikendraw/funcyou -q')

os.system('pip install funcyou/. -q')

fatal: destination path 'caption-generator' already exists and is not an empty directory.


0

In [2]:
!ls

 Evaluate.txt			   get_data.py
 __pycache__			   image-caption-generator-deep-learning.ipynb
'another model.py'		   libdevice.10.bc
 captiongenerator_notebook.ipynb   log
 checkpoints			   main.py
 code.txt			   model.py
 config.py			   preprocessing.py
 data				   train_notebook.ipynb
 embedding			   utils.py
 funcyou


In [3]:

import numpy as np
import pandas as pd
import random, math
import tensorflow as tf
import glob
import shutil
from zipfile import ZipFile
import datetime
import sys
from functools import cache
from pathlib import Path
from tqdm import tqdm
import regex as re

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from sklearn.model_selection import train_test_split


from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocessing
from tensorflow.keras.layers import (
    TextVectorization, Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense, Attention, MultiHeadAttention, Flatten, Dropout,
    Concatenate, Activation, GlobalAveragePooling2D
    )
from tensorflow.keras.layers import LSTM, Embedding, Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow import keras
from tensorflow.keras.utils import array_to_img, img_to_array
import string
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, TensorBoard
from model import LearningRateDecayCallback, get_model, masked_acc, masked_loss
from preprocessing import preprocess_text, embedding_matrix_creater, mapper, clean_words, clean_df
from utils import create_model_checkpoint

from config import config

from get_data import download_dataset
from funcyou.dataset import download_kaggle_dataset

from funcyou.utils import printt, dir_walk
import matplotlib.pyplot as plt
from nltk import word_tokenize
import nltk
from collections import Counter
import regex as re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


94668760/94668760 [==============================] - 4s 0us/step


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
seed_value = 12321
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)


BATCH_SIZE =    config.BATCH_SIZE
IMG_SIZE =      config.IMG_SIZE
CHANNELS =      config.CHANNELS
IMG_SHAPE =     config.IMG_SHAPE
MAX_LEN =       config.MAX_LEN
EPOCHS =        config.EPOCHS
LEARNING_RATE = config.LEARNING_RATE
UNITS =         config.UNITS
raw_caption_file =  config.raw_caption_file
caption_file =  config.caption_file
image_dir =     config.image_dir
glove_path =    config.glove_path
TEST_SIZE =     config.TEST_SIZE
VAL_SIZE=       config.VAL_SIZE
EMBEDDING_DIMENSION =   config.EMBEDDING_DIMENSION 

In [5]:
print(f'''
{BATCH_SIZE=}
{IMG_SIZE=}
{CHANNELS=}
{IMG_SHAPE=}
{MAX_LEN=}
{EPOCHS=}
{LEARNING_RATE=}
{UNITS=}
{raw_caption_file=}
{caption_file=}
{image_dir=}
{glove_path=}
{TEST_SIZE=}
{VAL_SIZE=}
{EMBEDDING_DIMENSION=}
''')


BATCH_SIZE=8
IMG_SIZE=256
CHANNELS=3
IMG_SHAPE=(256, 256, 3)
MAX_LEN=50
EPOCHS=10
LEARNING_RATE=0.01
UNITS=16
raw_caption_file=PosixPath('input/flickr30k/results.csv')
caption_file=PosixPath('input/flickr30k/results_cleaned.csv')
image_dir=PosixPath('input/flickr30k/images')
glove_path=PosixPath('embedding/glove.6B.50d.zip')
TEST_SIZE=0.05
VAL_SIZE=0.05
EMBEDDING_DIMENSION=50



I had this setting when i tried run in py computer. Now changing according to kaggle

In [6]:
# dd = dir_walk('/kaggle/input')
# dd

In [7]:
# dd.iloc[4]['directory']

In [8]:
caption_file = '/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv'
image_dir = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/flickr30k_images'

In [9]:
df = pd.read_csv(caption_file, sep = '|')

# column names have spaces in them
df.columns = df.columns.str.lower().str.strip()

# dropping some values at 19999
df.drop(19999, inplace = True) #bad value at 19999

df['comment_number'] = pd.to_numeric(df['comment_number'])

#removing nulls
df.isnull().sum()
df = df.dropna()


print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158914 entries, 0 to 158914
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   image_name      158914 non-null  object
 1   comment_number  158914 non-null  int64 
 2   comment         158914 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.8+ MB
None


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [10]:
# get all unique words by count
def word_count_df(x:pd.Series):
    text_data = x.str.lower().str.cat(sep=' ')
    words = word_tokenize(text_data) 
    word_count = Counter(words)
    unique_words = set(words)
    len(word_count.keys()), len(word_count.values())

    countdf = pd.DataFrame([word_count.keys(), word_count.values()]).T
    countdf.columns = ['word', 'counts']
    return countdf


In [11]:
countdf = word_count_df(df['comment'])
countdf.sort_values('counts', ascending = False).head()

,word,counts
29,a,271704
16,.,151059
13,in,83466
14,the,62978
51,on,45669


In [12]:
# removing low frequency words, they aren't contributing much
low_freq_words_df = countdf[countdf.counts<5]
print('low freq words : ',len(low_freq_words_df))
print(low_freq_words_df.word.values[:100])

low freq words :  12561
['spent' 'stitching' 'sequence' 'tractors' 'cruises' 'sifter' 'bundt'
 'fingerpaints' 'instrumentalists' 'curtained' 'portrayed' 'blitz'
 'toothpick' 'origami' 'justin' 'bieber' 'urinals' 'urinal' 'obliverate'
 'uniformly' 'colleges' 'meetings' 'checkpoint' 'advertise'
 'single-person' '4-wheeled' 'gamecube' 'picnicking' 'aross' 'mottled'
 'placards' 'belays' 'supple' 'sheared' 'spaceship' 'hutch' 'decipher'
 'tonight' 'questioning' 'frolicks' 'crampons' 'disregarding' 'slush'
 'snowmobiling' 'lucky' 'prescription' 'spectrum' 'month' 'sellers' 'jail'
 'band-aid' 'life-vest' 'black-colored' 'filler' 'lowly' 'entrees'
 'hesitates' 'suns' 'relatives' 'moutains' 'connects' 'tree-covered'
 'multiracial' 'bassoon' 'beanies' 'explained' 'somethings' 'betty' 'boop'
 'noddles' 'papayas' 'septic' 'oblong' 'whimsically' 'pantaloons'
 'cradled' 'relevance' 'cobs' 'sheath' 'unsheathes' 'riverbed' 'snapshot'
 'appetizers' 'pillow-fight' 'i-beam' 'blender' 'magizine' 'overshir

we are keeping words in comments which atleast has occured  5 times( there are unnecessary word which we don't want) 
there are ~12000 words that doesn't even gets repeated 5 times in 150000 lines

In [13]:
words_to_keep=set(countdf[countdf['counts']>4].word.values)
len(words_to_keep)

7732

In [14]:
# now removing words not in words_to_keep

def clean_words(x, words_to_keep):
    words = re.split(r'\W+', x)
    return ' '.join(w for w in words if w.lower() in words_to_keep)


In [15]:
df['comment'] = df['comment'].map(lambda x: clean_words(x,words_to_keep=words_to_keep))


In [16]:
# adding start and end token

START_TOKEN = 'startseq'
END_TOKEN = 'endseq'

df['comment'] = START_TOKEN + ' ' + df['comment'] + ' ' + END_TOKEN

# image_path
df['image_path'] = str(image_dir) + '/' + df['image_name']

# word_length
df['word_length'] = df['comment'].apply(lambda x: len(str(x).split()))


In [17]:
df['img_exists'] = df['image_path'].apply(lambda x: os.path.isfile(x))

In [18]:
df.head()

,image_name,comment_number,comment,image_path,word_length,img_exists
0,1000092795.jpg,0,startseq Two young guys with shaggy hair look ...,/kaggle/input/flickr-image-dataset/flickr30k_i...,18,True
1,1000092795.jpg,1,startseq Two young White males are outside nea...,/kaggle/input/flickr-image-dataset/flickr30k_i...,11,True
2,1000092795.jpg,2,startseq Two men in green shirts are standing ...,/kaggle/input/flickr-image-dataset/flickr30k_i...,12,True
3,1000092795.jpg,3,startseq A man in a blue shirt standing in a g...,/kaggle/input/flickr-image-dataset/flickr30k_i...,12,True
4,1000092795.jpg,4,startseq Two friends enjoy time together endseq,/kaggle/input/flickr-image-dataset/flickr30k_i...,7,True


### **For the Sake of COmputation power and time we will train with one caption per image rather than 5 caption per image**

In [19]:
df.shape

(158914, 6)

In [20]:
# i randomly chose 2nd comment of all picture you can chose anything bw 0 to 4
df= df[(df.comment_number==2) & (df.img_exists==True)]
df.shape 

(31783, 6)

now we are just using 1/5 dataset

# Tokenize

In [21]:

#tokenizer
tokenizer = TextVectorization(standardize=preprocess_text)
tokenizer.adapt(df['comment'])


word_to_id = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
id_to_word = tf.keras.layers.StringLookup(vocabulary=tokenizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)


In [22]:
# load image model
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=tf.keras.layers.Input(shape=IMG_SHAPE))

resnet.trainable = False

# making Dataset

In [23]:
def preprocess_text(text):

    text = tf.strings.lower(text)

    text = tf.strings.regex_replace(text, r'\d', '')

    # Remove any punctuations
    text = tf.strings.regex_replace(text, '[%s]' % re.escape(
        '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'), '')

    # Remove single characters
    text = tf.strings.regex_replace(text, r'\b\w\b', '')
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    return text

def mapper(x, y, tokenizer):
    x = load_images_now(x)
    y = tokenizer(y)

    y_in = y[:-1]
    y_in =  tf.pad(y_in, [[0, MAX_LEN - tf.shape(y_in)[0]]] , constant_values=0)

    y_out = y[1:]
    y_out =  tf.pad(y_out, [[0, MAX_LEN - tf.shape(y_out)[0]]], constant_values=0)

    return (x, y_in), y_out


@tf.function
def load_images_now(x):
    image_data = tf.io.read_file(x)
    image_features = tf.image.decode_jpeg(image_data, channels=CHANNELS)
    image_features = tf.image.resize_with_pad(
        image_features, target_height=IMG_SIZE, target_width=IMG_SIZE)
    image_features = tf.keras.applications.resnet.preprocess_input(
        image_features)
    image_features = tf.reshape(
        image_features, (1, IMG_SIZE, IMG_SIZE, CHANNELS))
    image_features = resnet(image_features)
    image_features = GlobalAveragePooling2D()(image_features)
    image_features = tf.squeeze(image_features)

    return image_features


In [24]:
# Creating dataset
TEST_SIZE = config.TEST_SIZE
VAL_SIZE =  config.VAL_SIZE

train, val = train_test_split(
    df[['image_path', 'comment']],  test_size=VAL_SIZE, random_state=11)
train, test = train_test_split(
    train[['image_path', 'comment']],  test_size=TEST_SIZE, random_state=11)


train_data = tf.data.Dataset.from_tensor_slices((train.image_path, train.comment))
test_data = tf.data.Dataset.from_tensor_slices((test.image_path, test.comment))
val_data = tf.data.Dataset.from_tensor_slices((val.image_path, val.comment))


train_data = train_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data =   test_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data =     val_data.map(lambda x,y:mapper(x, y, tokenizer)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# resnet_output_flattened_shape = 8*8*2048

print("Number of training samples: %d" %
      tf.data.experimental.cardinality(train_data))
print("Number of validation samples: %d" %
      tf.data.experimental.cardinality(val_data))
print("Number of test samples: %d" %
      tf.data.experimental.cardinality(test_data))

VOCAB_SIZE = tokenizer.vocabulary_size()
print("Vocabulary size: %d" % VOCAB_SIZE)


Number of training samples: 3586
Number of validation samples: 199
Number of test samples: 189
Vocabulary size: 6830


In [25]:
train.shape, len(train_data)*8

((28683, 2), 28688)

In [26]:
for (img_in, txt_in), txt_out in train_data.take(1):
    # print(f'{i.numpy().decode():<40} {j.numpy()}')
    print('x     : ', img_in.shape)
    print('y_in  : ', txt_in)
    print('y_out : ', txt_out)
    print('\n')


x     :  (8, 2048)
y_in  :  tf.Tensor(
[[   2   31    8   26    6    5  429   10   65  352  174    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   2   45    9    7  228    4  254   16  399   59  297    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   2   12    4   87  396  129  536    6   40  163   38   36 1244  128
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   2   18  232   12  118   36    5  669   10  223  106    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    

# Model

In [27]:

def get_model():
    encoder = LSTM(UNITS, return_sequences=True, return_state=True)
    decoder = LSTM(UNITS, return_sequences=True)

    embedding = Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=EMBEDDING_DIMENSION,
        mask_zero=True,
        input_length=MAX_LEN,
        trainable=True,
    )



    image_input = Input(shape=(2048,), name = 'image_input')
    print('image_input: ',image_input.shape)

    x = Dropout(.3)(image_input)
    print('enc dropout: ',x.shape)
    
    x = Dense(MAX_LEN*UNITS, activation = 'relu', kernel_initializer = 'glorot_uniform' )(x)
    print('Dense: ',x.shape)
    
    x = tf.reshape(x, (-1, MAX_LEN, UNITS))
    print('reshape: ',x.shape)


    txt_input = Input(shape=(MAX_LEN,), name = 'text_input')
    print('txt_input: ',txt_input.shape)
    
    i = embedding(txt_input)
    print('text_embedding: ',i.shape)
    
    i, j, k = encoder(i)
    print('encoder output: ',i.shape)
    
    i = Dropout(.3)(i)
    print('enc dropout: ',i.shape)
    
    i = decoder(i, initial_state=[j, k])
    print('decoder output: ',i.shape)
    
    i = Dropout(.3)(i)
    print('decoder dropout: ',i.shape)

    l = Attention()([x, i])
    print('attention: ',l.shape)

    m = Concatenate()([i, l])
    print('concat: ', m.shape)

    m = Dropout(.3)(m)
    print('concat dropout: ',m.shape)

    m = Dense(MAX_LEN*UNITS)(m)
    print('Dense1: ', m.shape)


    m = Dense(VOCAB_SIZE)(m)
    print('Dense3_final: ', m.shape)
    return Model(inputs=[image_input, txt_input], outputs=m)


In [28]:
model = get_model()
print(model.summary())

image_input:  (None, 2048)
enc dropout:  (None, 2048)
Dense:  (None, 800)
reshape:  (None, 50, 16)
txt_input:  (None, 50)
text_embedding:  (None, 50, 50)
encoder output:  (None, 50, 16)
enc dropout:  (None, 50, 16)
decoder output:  (None, 50, 16)
decoder dropout:  (None, 50, 16)
attention:  (None, 50, 16)
concat:  (None, 50, 32)
concat dropout:  (None, 50, 32)
Dense1:  (None, 50, 800)
Dense3_final:  (None, 50, 6830)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 50)       341500      ['text_input[0][0]']             
                                                                      

In [29]:

def masked_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        reduction='none')
    loss = loss_fn(y_true, y_pred)

    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    matchh = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(matchh)/tf.reduce_sum(mask)


In [30]:
print(LEARNING_RATE)

0.01


In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=masked_loss,
              metrics=[masked_acc])


In [32]:
os.makedirs('log', exist_ok=True)
csv_logger = CSVLogger('./log/training.log')
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)

In [33]:
EPOCHS = 60

print(len(train_data) // EPOCHS, len(val_data) // EPOCHS)

steps_per_epoch = int(1*(len(train_data) / EPOCHS))
validation_steps =  int(1*(len(val_data) / EPOCHS))
print(steps_per_epoch, validation_steps)

steps_per_epoch = 150
validation_steps = 2


59 3
59 3


## model.fit

In [34]:
history = model.fit(train_data.repeat(),
                    epochs=EPOCHS,
                    validation_data=val_data,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[
                        # decay_callback,
                        csv_logger, create_model_checkpoint(model_name = 'capgen', save_dir = 'checkpoints', monitor = 'masked_acc')
                                ]
                    )

Epoch 1/60
150/150 [==============================] - 198s 1s/step - loss: 9.9848 - masked_acc: 0.0076 - val_loss: 8.8291 - val_masked_acc: 0.0000e+00
Epoch 2/60
150/150 [==============================] - 146s 976ms/step - loss: 8.8291 - masked_acc: 8.4720e-04 - val_loss: 8.8291 - val_masked_acc: 0.0000e+00
Epoch 3/60
150/150 [==============================] - 152s 1s/step - loss: 8.8291 - masked_acc: 5.3447e-04 - val_loss: 8.8291 - val_masked_acc: 0.0000e+00
Epoch 4/60
102/150 [===================>..........] - ETA: 48s - loss: 8.8291 - masked_acc: 3.3769e-04

KeyboardInterrupt: 

In [ ]:
from datetime import datetime

In [ ]:
model.save(f'saved_model/{datetime.now()}-{EPOCH}.h5')

In [ ]:
len(test_data)